## Day 13: Shuttle Search 

Your ferry can make it safely to a nearby port, but it won't get much further. When you call to book another ship, you discover 
that no ships embark from that port to your vacation island. You'll need to get from the port to the nearest airport.

Fortunately, a shuttle bus service is available to bring you from the sea port to the airport! Each bus has an ID number that also indicates how 
often the bus leaves for the airport.

Bus schedules are defined based on a timestamp that measures the number of minutes since some fixed reference point in the past. At timestamp 0, 
every bus simultaneously departed from the sea port. After that, each bus travels to the airport, then various other locations, and finally returns to 
the sea port to repeat its journey forever.

The time this loop takes a particular bus is also its ID number: the bus with ID 5 departs from the sea port at timestamps 0, 5, 10, 15, and so on. The 
bus with ID 11 departs at 0, 11, 22, 33, and so on. If you are there when the bus departs, you can ride that bus to the airport!

Your notes (your puzzle input) consist of two lines. The first line is your estimate of the earliest timestamp you could depart on a bus. The second line 
lists the bus IDs that are in service according to the shuttle company; entries that show x must be out of service, so you decide to ignore them.

To save time once you arrive, your goal is to figure out the earliest bus you can take to the airport. (There will be exactly one such bus.)

For example, suppose you have the following notes:
```
939
7,13,x,x,59,x,31,19
```

Here, the earliest timestamp you could depart is 939, and the bus IDs in service are 7, 13, 59, 31, and 19. Near timestamp 939, these bus 
IDs depart at the times marked D:
```
time   bus 7   bus 13  bus 59  bus 31  bus 19
929      .       .       .       .       .
930      .       .       .       D       .
931      D       .       .       .       D
932      .       .       .       .       .
933      .       .       .       .       .
934      .       .       .       .       .
935      .       .       .       .       .
936      .       D       .       .       .
937      .       .       .       .       .
938      D       .       .       .       .
939      .       .       .       .       .
940      .       .       .       .       .
941      .       .       .       .       .
942      .       .       .       .       .
943      .       .       .       .       .
944      .       .       D       .       .
945      D       .       .       .       .
946      .       .       .       .       .
947      .       .       .       .       .
948      .       .       .       .       .
949      .       D       .       .       .
```
The earliest bus you could take is bus ID 59. It doesn't depart until timestamp 944, so you would need to wait 944 - 939 = 5 minutes before it departs. Multiplying 
the bus ID by the number of minutes you'd need to wait gives 295.

** What is the ID of the earliest bus you can take to the airport multiplied by the number of minutes you'll need to wait for that bus?**

In [1]:
from numpy import prod
from math import gcd
import copy

In [2]:
puzzle_input = list(open('../data/day13_input.txt', 'r'))
data = [row.strip() for row in puzzle_input]

In [3]:
example_input = list(open('../examples/day13_example.txt', 'r'))
example = [row.strip() for row in example_input]

In [4]:
def part1(d):
    times = [int(time) for time in d[1].split(',') if time !='x']
    print(times)
    arrival_time = int(d[0])
    differences = []
    for i, time in enumerate(times):
        differences.append(time - arrival_time%time)
    print(differences)
    index_of_interest = differences.index(min(differences))
    print(f'The answer to part 1 is {times[index_of_interest] * differences[index_of_interest]}')

In [5]:
part1(data)

[29, 41, 601, 23, 13, 17, 19, 463, 37]
[12, 31, 8, 18, 9, 13, 18, 398, 18]
The answer to part 1 is 4808


## Part Two 

The shuttle company is running a contest: one gold coin for anyone that can find the earliest timestamp such that the first bus ID 
departs at that time and each subsequent listed bus ID departs at that subsequent minute. (The first line in your input is no longer relevant.)

For example, suppose you have the same list of bus IDs as above:

7,13,x,x,59,x,31,19

An x in the schedule means there are no constraints on what bus IDs must depart at that time.

This means you are looking for the earliest timestamp (called t) such that:
```
    Bus ID 7 departs at timestamp t.
    Bus ID 13 departs one minute after timestamp t.
    There are no requirements or restrictions on departures at two or three minutes after timestamp t.
    Bus ID 59 departs four minutes after timestamp t.
    There are no requirements or restrictions on departures at five minutes after timestamp t.
    Bus ID 31 departs six minutes after timestamp t.
    Bus ID 19 departs seven minutes after timestamp t.
```
The only bus departures that matter are the listed bus IDs at their specific offsets from t. Those bus IDs can depart at other times, and other bus IDs can depart at those times. For example, in the list above, because bus ID 19 must depart seven minutes after the timestamp at which bus ID 7 departs, bus ID 7 will always also be departing with bus ID 19 at seven minutes after timestamp t.

In this example, the earliest timestamp at which this occurs is 1068781:
```
time     bus 7   bus 13  bus 59  bus 31  bus 19
1068773    .       .       .       .       .
1068774    D       .       .       .       .
1068775    .       .       .       .       .
1068776    .       .       .       .       .
1068777    .       .       .       .       .
1068778    .       .       .       .       .
1068779    .       .       .       .       .
1068780    .       .       .       .       .
1068781    D       .       .       .       .
1068782    .       D       .       .       .
1068783    .       .       .       .       .
1068784    .       .       .       .       .
1068785    .       .       D       .       .
1068786    .       .       .       .       .
1068787    .       .       .       D       .
1068788    D       .       .       .       D
1068789    .       .       .       .       .
1068790    .       .       .       .       .
1068791    .       .       .       .       .
1068792    .       .       .       .       .
1068793    .       .       .       .       .
1068794    .       .       .       .       .
1068795    D       D       .       .       .
1068796    .       .       .       .       .
1068797    .       .       .       .       .
```
In the above example, bus ID 7 departs at timestamp 1068788 (seven minutes after t). This is fine; the only requirement on that minute is that bus ID 19 departs then, and it does.

Here are some other examples:
```
    The earliest timestamp that matches the list 17,x,13,19 is 3417.
    67,7,59,61 first occurs at timestamp 754018.
    67,x,7,59,61 first occurs at timestamp 779210.
    67,7,x,59,61 first occurs at timestamp 1261476.
    1789,37,47,1889 first occurs at timestamp 1202161486.
```
However, with so many bus IDs in your list, surely the actual earliest timestamp will be larger than 100000000000000!

**What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?**


I saw someone [reply](https://twitter.com/pietroppeter/status/1338020748837924864) on one [Eric Wastl](https://twitter.com/ericwastl)'s (creator of Advent of Code's) twitter posts, "Yeah, sometimes a day may look like Chinese to you, but the remainder of days might not..."
on December 13th which totally makes the connection to the [Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem). To see the statement of the theorem, go to the previous link.  I remember these problems being a bit tedious to solve by hand, even with small cases.

Let's look at our first example, and how we might solve this using the Chinese Remainder Theorem. Note: We can use the CRT here, because all of our modulo 
numbers are coprime.

We have the bus ids: $[m_0, m_1, ..., m_7] = [7,13,x,x,59,x,31,19]$. An $x$ in the schedule means there are no constraints on what bus IDs must depart at that time. The problem 
states that we are looking for the smallest $T$ such that:
- $T~~ modulo ~~7 = 0 ~~\implies T \equiv 0~ (mod~ 7)$
- $T+1~~ modulo ~~13 = 0 \implies T \equiv 12~ (mod~ 13)$
- $T+4~~ modulo ~~59 = 0 \implies T \equiv 55~ (mod~ 59)$
- $T+6~~ modulo ~~31 = 0 \implies T \equiv 25~ (mod~ 31)$
- $T+7~~ modulo ~~19 = 0 \implies T \equiv 12~ (mod~ 19)$

There are five steps to this algorithm:
- Step 1. Create list $m=[m_1,m_2,...,m_j]$ of the modulo numbers.
    - Ex: $m = [7,13,59,31,19]$
- Step 2. Construct $M = m_1 * m_2 * ... * m_j$
    - Ex: $M = 7 * 13 * 59 * 31 * 19 = 3162341$ (upper bound on our answer, by the way!)
- Step 3. Find $n_i = \displaystyle \prod_{k\neq i}^{j} m_{k}$ for each $m_k$ in $m$
    - Ex: $n_1 = 13 * 59 * 31 * 19 = 451763$
- Step 4. Use [Euclidean Algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Modular_integers) to solve for $n_i y_i \equiv 1~ (mod~ m_i)$.
    - Side note: I hadn't thought about this before, but the Euclidean Algorithm is a really simple recursion problem.
    - Ex: Find $451763y_1 \equiv 1~(mod~ 7)$
        - $451763 = 64537 * 7 + 4$
        - $64537 = 9219 * 7 + 4$
        - $9219 = 1317 * 7 + 0 \implies y_1 \equiv 0~(mod~ 7)$
    - I will end up just writing up a little function that does this Euclidean algorithm, it should be a 4-5 liner.
- Step 5. The smallest value for $\displaystyle T = \sum_{i = 1}^{j} m_i * n_i * y_i = m_1 * n_1 * y_1 + m_2 * n_2 * y_2 + ... + m_j * n_j * y_j$.


In [6]:
def euclidean_algorithm(a,b):
    if b == 0:
        return a
    else:
        return euclidean_algorithm(b, a % b)

In [7]:
def modularinverse(a, b):
    a = a % b; 
    for x in range(1, b) : 
        if ((a * x) % b == 1) : 
            return x 
    return 1

In [8]:
euclidean_algorithm(32,6)

2

In [9]:
def part2(d):
    m_list = [int(time) for time in d[1].split(',') if time !='x'] # Step 1
    print(m_list)
    n_list = [] # Initiate empty list for n_i's
    y_list = [] # Initiate empty list for y_i's
    all_list = m_list.copy() # For final product, start with the m_list, add on the other lists later
    M = prod(m_list) # Step 2 
    print(f'M is {M}')
    for index, m in enumerate(m_list): # This loop is Step 3
        n_list.append(int(M//m))
    print(f'n_list is {n_list}')
    all_list.extend(n_list)
    for index, n in enumerate(n_list): # This loop is Step 4
        print(f'n is {n}')
        print(f'm_list[index] is {m_list[index]}')
        y_i = pow(n, -1, m_list[index])
        print(f'y_i is {y_i}')
        y_list.append(y_i)
    print(f'ylist is {y_list}')
    all_list.extend(y_list)
    print(all_list)
    solution = 0
    for i in range(len(m_list)):
        print(f'i is {i}')
        p = 1
        for j in range(len(all_list)):
            if j % len(m_list) == i:
                p*=all_list[i]
        print(f'p is {p}')
        solution = (solution + p) #I feel like here I need to multiply, not add, but that's giving zero...
    return(solution % M)

In [10]:
part2(example)

[7, 13, 59, 31, 19]
M is 3162341
n_list is [451763, 243257, 53599, 102011, 166439]
n is 451763
m_list[index] is 7
y_i is 2
n is 243257
m_list[index] is 13
y_i is 1
n is 53599
m_list[index] is 59
y_i is 35
n is 102011
m_list[index] is 31
y_i is 3
n is 166439
m_list[index] is 19
y_i is 18
ylist is [2, 1, 35, 3, 18]
[7, 13, 59, 31, 19, 451763, 243257, 53599, 102011, 166439, 2, 1, 35, 3, 18]
i is 0
p is 343
i is 1
p is 2197
i is 2
p is 205379
i is 3
p is 29791
i is 4
p is 6859


244569

In [11]:
part2(data)

[29, 41, 601, 23, 13, 17, 19, 463, 37]
M is 1182259336403743
n_list is [40767563324267, 28835593570823, 1967153637943, 51402579843641, 90943025877211, 69544666847279, 62224175600197, 2553475888561, 31952955037939]
n is 40767563324267
m_list[index] is 29
y_i is 8
n is 28835593570823
m_list[index] is 41
y_i is 40
n is 1967153637943
m_list[index] is 601
y_i is 20
n is 51402579843641
m_list[index] is 23
y_i is 21
n is 90943025877211
m_list[index] is 13
y_i is 1
n is 69544666847279
m_list[index] is 17
y_i is 7
n is 62224175600197
m_list[index] is 19
y_i is 3
n is 2553475888561
m_list[index] is 463
y_i is 22
n is 31952955037939
m_list[index] is 37
y_i is 4
ylist is [8, 40, 20, 21, 1, 7, 3, 22, 4]
[29, 41, 601, 23, 13, 17, 19, 463, 37, 40767563324267, 28835593570823, 1967153637943, 51402579843641, 90943025877211, 69544666847279, 62224175600197, 2553475888561, 31952955037939, 8, 40, 20, 21, 1, 7, 3, 22, 4]
i is 0
p is 24389
i is 1
p is 68921
i is 2
p is 217081801
i is 3
p is 12167
i is 4
p is 

316504747

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=320b6760-c2f1-4eb8-a5b7-b5aca5d8cf36' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>